In [11]:
import numpy as np
from stable_baselines3 import PPO
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
from environments import MultiMarkEnv

In [12]:
outer_radius = 2 * 250 + 2.5 * 0.1 * 250

dt = 5
num_marks = 2
max_seconds = 500 * num_marks / dt

# Initialize the figure and scatter plot
fig = go.FigureWidget()
scatter = fig.add_scatter(mode='markers+lines', name='trajectory')
marks_scatter = fig.add_scatter(mode='markers+text', name='marks')
fig.update_xaxes(range=[-outer_radius, outer_radius],dtick=25)
fig.update_yaxes(range=[-outer_radius, outer_radius],dtick=25)
fig.layout.width=800
fig.layout.height=800

heading_fig = go.FigureWidget()
heading_scatter = heading_fig.add_scatter(mode='markers+lines', name='heading')
rudder_angle_scatter = heading_fig.add_scatter(mode='markers+lines', name='rudder angle')
heading_fig.update_xaxes(range=[0, max_seconds],dtick=25)
heading_fig.update_yaxes(range=[-210, 210],dtick=30)
heading_fig.layout.width=800
heading_fig.layout.height=600
heading_fig.layout.title = "Heading over time"

vmg_fig = go.FigureWidget()
vmg_scatter = vmg_fig.add_scatter(mode='markers+lines')
vmg_fig.update_xaxes(range=[0, max_seconds],dtick=25)
vmg_fig.update_yaxes(range=[-4, 4],dtick=2)
vmg_fig.layout.width=800
vmg_fig.layout.height=400
vmg_fig.layout.title = "VMG over time"

# Initialize output for the text
out = widgets.Output()

# Display text and figure
display(out)
display(fig)
display(heading_fig)
display(vmg_fig)

episode = 0

colormap = ['blue', 'red', 'green', 'orange', 'black']

def plot(data, marks, bounds):
    global out, fig
    x_values = [point['x'] for point in data]
    y_values = [point['y'] for point in data]
    meta_values = [{k: '%.3f' % v for (k, v) in point['meta'].items()} for point in data]

    min_x, max_x, min_y, max_y = bounds

    color_meta = [point['meta']['current_mark'] for point in data]
    colors = [colormap[color % len(colormap)] for color in color_meta]

    headings = [point['meta']['heading'] for point in data]
    rudder_angles = [point['meta']['rudder_angle'] for point in data]
    vmg = [point['meta']['vmg'] for point in data]

    global_meta = {'final_position': (x_values[-1], y_values[-1]), 'reward': sum([point['meta']['reward'] for point in data]), 'iters': len(data), 'episode': episode}

    # Update data
    scatter = fig.data[0]
    scatter.x = x_values
    scatter.y = y_values
    scatter.hovertext = meta_values  # Add this line
    scatter.marker.color = colors
    scatter.line.color = 'lightgrey'

    marks_scatter = fig.data[1]
    marks_scatter.x = [x for x, _ in marks]
    marks_scatter.y = [y for _, y in marks]
    marks_scatter.text = list(range(len(marks)))
    marks_scatter.textposition = 'bottom right'
    marks_scatter.marker.color = 'black'
    marks_scatter.marker.symbol = 'x'
    marks_scatter.marker.size = 10

    heading_scatter = heading_fig.data[0]
    heading_scatter.x = list(range(len(headings)))
    heading_scatter.y = headings
    heading_scatter.marker.color = colors
    heading_scatter.line.color = 'lightgrey'

    rudder_angle_scatter = heading_fig.data[1]
    rudder_angle_scatter.x = list(range(len(rudder_angles)))
    rudder_angle_scatter.y = rudder_angles
    rudder_angle_scatter.marker.color = 'lightgreen'
    rudder_angle_scatter.line.color = 'lightgreen'

    vmg_scatter = vmg_fig.data[0]
    vmg_scatter.x = list(range(len(vmg)))
    vmg_scatter.y = vmg
    vmg_scatter.marker.color = colors
    vmg_scatter.line.color = 'lightgrey'

    fig.update_xaxes(range=[min_x * 1.1, max_x * 1.1],dtick=25)
    fig.update_yaxes(range=[min_y * 1.1, max_y * 1.1],dtick=25)
    fig.layout.width=800
    fig.layout.height=800

    heading_fig.update_yaxes(range=[-210, 210],dtick=30)
    heading_fig.layout.width=800
    heading_fig.layout.height=600

    vmg_fig.layout.width=800
    vmg_fig.layout.height=400

    # Update the text output
    with out:
        out.clear_output(wait=True)
        print(f"Global Meta: {str(global_meta)}")

None

Output()

FigureWidget({
    'data': [{'mode': 'markers+lines',
              'name': 'trajectory',
              'type': 'scatter',
              'uid': 'abee810d-6d78-4665-bfef-a7005a5c5716'},
             {'mode': 'markers+text', 'name': 'marks', 'type': 'scatter', 'uid': '1ea6e92e-ea93-412f-958b-2ae9c21977ef'}],
    'layout': {'height': 800,
               'template': '...',
               'width': 800,
               'xaxis': {'dtick': 25, 'range': [-562.5, 562.5]},
               'yaxis': {'dtick': 25, 'range': [-562.5, 562.5]}}
})

FigureWidget({
    'data': [{'mode': 'markers+lines',
              'name': 'heading',
              'type': 'scatter',
              'uid': '3d5fa1cb-703a-415d-a096-cfcbd3ee0942'},
             {'mode': 'markers+lines',
              'name': 'rudder angle',
              'type': 'scatter',
              'uid': '64395441-6b1f-4d2c-a546-8d24177fe4c6'}],
    'layout': {'height': 600,
               'template': '...',
               'title': {'text': 'Heading over time'},
               'width': 800,
               'xaxis': {'dtick': 25, 'range': [0, 200.0]},
               'yaxis': {'dtick': 30, 'range': [-210, 210]}}
})

FigureWidget({
    'data': [{'mode': 'markers+lines', 'type': 'scatter', 'uid': 'cec60a8b-e8ed-4a3c-910d-6717a56eec4f'}],
    'layout': {'height': 400,
               'template': '...',
               'title': {'text': 'VMG over time'},
               'width': 800,
               'xaxis': {'dtick': 25, 'range': [0, 200.0]},
               'yaxis': {'dtick': 2, 'range': [-4, 4]}}
})

In [13]:
# Configuration

r = 250
# actions = np.array([-90, -45, -15, 0, 15, 45, 90]) / 180.0
actions = np.array([-90, -45, 0, 45, 90]) / 180.0

config = {
    'max_marks': 1,
    'max_seconds_per_leg': 500,
    'plot_fn': lambda x, y, z: None,
    'leg_radius': r,
    'actions': actions,
    'target_tolerance_multiplier': 2
}

outer_radius = config['max_marks'] * 2 * r + 0.2 * r
bounds = [-outer_radius, outer_radius, -outer_radius, outer_radius]
# Initialize Environment
env = MultiMarkEnv(config, dt=5, bounds=bounds, seq_size=1)

# Initialize PPO model
model = PPO.load('./BoatControl_boat_model_fine.dat')

In [14]:
targets = [(0, 250), (250, 125), (250, 0), (0, 0), (-125, 0), (-125, -125), (0, -125), (0, 0)]

full_trajectory = []

start_x = 0
start_y = 0
start_heading = np.pi/4
start_vmg = 0

for mark_idx, (target_x, target_y) in enumerate(targets):
    best_trajectory = None
    best_reward = None
    for i in range(10):
        obs, _ = env.reset(heading=start_heading, vmg=start_vmg)
        env.target_x[0] = target_x - start_x
        env.target_y[0] = target_y - start_y
        done = False
        total_reward = 0
        while not done:
            action, _states = model.predict(obs)
            obs, reward, is_terminal, is_truncated, info = env.step(action)
            done = is_terminal or is_truncated
            total_reward += reward

        if best_reward is None or total_reward > best_reward:
            best_reward = total_reward
            best_trajectory = env.trajectory

    for item in best_trajectory:
        item['x'] += start_x
        item['y'] += start_y
        item['meta']['current_mark'] = mark_idx

    full_trajectory.extend(best_trajectory)
    start_x = best_trajectory[-1]['x']
    start_y = best_trajectory[-1]['y']
    start_heading = env.boat.heading
    start_vmg = best_trajectory[-1]['meta']['vmg']

plot(full_trajectory, targets, bounds)

In [15]:
full_trajectory

[{'x': 0.0,
  'y': 0.0,
  'meta': {'current_mark': 0,
   'vmg': 0,
   'heading': 45.0,
   'rudder_angle': 90.0,
   'reward': 0,
   'speed': 0}},
 {'x': 12.43919180445222,
  'y': 8.61046850073517,
  'meta': {'current_mark': 0,
   'vmg': 1.255267769722253,
   'heading': 63.95876613156316,
   'rudder_angle': 90.0,
   'reward': 0.1255267769722253,
   'speed': array(8.66531435)}},
 {'x': 28.219994085617923,
  'y': 11.941072213297268,
  'meta': {'current_mark': 0,
   'vmg': -0.2435591004449742,
   'heading': 87.50771374776131,
   'rudder_angle': 90.0,
   'reward': -0.02435591004449742,
   'speed': array(9.49222984)}},
 {'x': 43.603921640191224,
  'y': 16.550735486683124,
  'meta': {'current_mark': 0,
   'vmg': 0.8591240297259978,
   'heading': 63.851312352916835,
   'rudder_angle': -90.0,
   'reward': 0.08591240297259978,
   'speed': array(8.12946241)}},
 {'x': 56.86987382880187,
  'y': 25.199046791944276,
  'meta': {'current_mark': 0,
   'vmg': 1.2585630578950529,
   'heading': 52.262909601